In [ ]:
# default_exp preview_folder
# default_cls_lvl 2

# Preview Folder
> This model takes pictures every x minutes and saves them in a separate folder and also organizes pictures in different folders

In [ ]:
#hide
from nbdev.showdoc import *

## Imports

In [ ]:
#export
import os
import shutil
from datetime import datetime
from pathlib import Path
import pandas as pd

In [ ]:
#export
def convert_path_timestamp(path:[str,Path]):
    """converts a filename to timestamp"""
    path = Path(path)
    if path.is_file() is False:
        return None
    
    filename = path.stem
    return datetime.strptime(filename[:14], '%Y%m%d%H%M%S')

In [ ]:
path = Path('data/to_label/20200320/20200320062300_006411.jpg')
assert type(convert_path_timestamp(path)) is datetime

In [ ]:
def get_filename_df(path_folder:[str, Path]):
    """converts image filepaths to a pandas df"""
    path_folder = Path(path_folder)
    paths = list(path_folder.glob('**/*.jpg'))
    dates = [int(path.stem[:8]) for path in paths]
    hour = [int(path.stem[8:10]) for path in paths]
    datehour = [int(path.stem[:10]) for path in paths]
    paths_times = [convert_path_timestamp(path) for path in paths]
    df = pd.DataFrame(data={"path":paths, "date":dates, "hour":hour, "datehour": datehour}, index=paths_times)
    df = df.sort_index()
    return df
    

In [ ]:
df = get_filename_df('data/to_label/20200320/')
print(df)
assert type(df) == pd.DataFrame
assert 'datehour' in df.columns

In [ ]:
def create_datehour_folders(path:[str, Path], df:pd.DataFrame):
    """create a folder for every datehour combination in the dataframe"""
    assert 'datehour' in df.columns
    path = Path(path)
    for datehour in df.datehour.unique():
        print(f"create folder {datehour}")
        (path / str(datehour)).mkdir(exist_ok=True)

In [ ]:
path = Path('data/to_label/')
create_datehour_folders(path, df)

for datehour in df.datehour.unique():
    t_path = path / str(datehour)
    assert (t_path).exists()
    t_path.rmdir()
        

In [ ]:
def move_files(df:pd.DataFrame, path:[str, Path]):
    """move files from source folder to target folder"""
    path = Path(path)
    for index, row in df.iterrows():
        path_source = Path(row['path'])
        path_dest = path / str(row['datehour']) / path_source.name
        shutil.move(str(path_source), str(path_dest))
        #print(f"move from {path_source} to {path_dest}")
        
def remove_folder(path:[str, Path]):
    """removes folder and all files and folders within"""
    path = Path(path)
    for child in path.glob('*'):
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    path.rmdir()
        
def to_hour_folder(path_source:[str, Path]):
    """moves files from daily folder to hourly folders"""
    path_source = Path(path_source)
    df = get_filename_df(path_source)
    create_datehour_folders(path_source.parent, df)
    move_files(df, path_source.parent)
    remove_folder(path_source)

In [ ]:
!ls data/to_label/


In [ ]:
to_hour_folder('data/to_label/20200328/')
to_hour_folder('data/to_label/20200329/')
to_hour_folder('data/to_label/20200330/')
to_hour_folder('data/to_label/20200331/')
to_hour_folder('data/to_label/20200401/')
to_hour_folder('data/to_label/20200402/')
to_hour_folder('data/to_label/20200403/')

## Export

In [ ]:
from nbdev.export import *
notebook2script()